In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Pasamos el fichero a una lista (una línea por item)
with open('words.txt') as f:
    dictionary = f.read().splitlines()

In [5]:
vectorizer = CountVectorizer(binary= False, vocabulary=dictionary, stop_words='english', ngram_range = (1,1))

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
dir(train_data)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [8]:
train_data.target[0:100]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4,  8, 19,  4, 14,  6,  0,  1,
        7, 12,  5,  0, 10,  6,  2,  4,  1, 12,  9, 15,  7,  6, 13, 12, 17,
       18, 10,  8, 11,  8, 16,  9,  4,  3,  9,  9,  4,  4,  8, 12, 14,  5,
       15,  2, 13, 17, 11,  7, 10,  2, 14, 12,  5,  4,  6,  7,  0, 11, 16,
        0,  6, 17,  7, 12,  7,  3, 12, 11,  7,  2,  2,  0, 16,  1,  2,  7,
        3,  2,  1, 10, 12, 12, 17, 12,  2,  8,  8, 18,  5,  0,  1])

In [9]:
set(train_data.target)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

In [10]:
set(train_data.target_names)

{'alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc'}

Esto es para saber que hay suficientes elementos de cada tipo, por lo que podremos hacer selecciones aleatorias hasta tener 3 de cada

In [11]:
import numpy as np

In [12]:
unique, counts = np.unique(train_data.target, return_counts=True)
dict(zip(unique, counts))

{0: 480,
 1: 584,
 2: 591,
 3: 590,
 4: 578,
 5: 593,
 6: 585,
 7: 594,
 8: 598,
 9: 597,
 10: 600,
 11: 595,
 12: 591,
 13: 594,
 14: 593,
 15: 599,
 16: 546,
 17: 564,
 18: 465,
 19: 377}

In [13]:
train_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [15]:
train_vector_data = vectorizer.fit_transform(train_data.data)

In [16]:
print(train_vector_data)

  (0, 4151)	1
  (0, 52160)	1
  (0, 53807)	1
  (0, 56907)	1
  (0, 78575)	1
  (0, 112523)	1
  (0, 112565)	1
  (0, 124009)	1
  (0, 124372)	1
  (0, 149051)	1
  (0, 177802)	1
  (0, 184493)	1
  (0, 185287)	1
  (0, 190955)	1
  (0, 208543)	1
  (0, 213043)	1
  (0, 218699)	1
  (0, 221810)	1
  (0, 221819)	1
  (0, 225798)	1
  (0, 242690)	1
  (0, 251793)	1
  (0, 271493)	1
  (0, 306014)	1
  (0, 314710)	1
  :	:
  (11313, 12051)	1
  (11313, 31047)	1
  (11313, 110477)	1
  (11313, 124009)	1
  (11313, 163078)	1
  (11313, 170696)	1
  (11313, 192874)	1
  (11313, 217697)	1
  (11313, 218699)	1
  (11313, 239824)	1
  (11313, 265072)	2
  (11313, 271493)	1
  (11313, 306014)	1
  (11313, 340787)	1
  (11313, 359935)	1
  (11313, 360545)	1
  (11313, 365530)	1
  (11313, 385115)	2
  (11313, 389038)	1
  (11313, 391937)	1
  (11313, 398618)	1
  (11313, 400462)	1
  (11313, 410906)	1
  (11313, 413140)	1
  (11313, 419956)	1


In [17]:
feature_names = vectorizer.get_feature_names()

In [18]:
print(feature_names[265072])

number


In [19]:
write_terms(feature_names, None, train_vector_data, 1)

Mensaje 1 vectorizado: ['acceleration' 'adapters' 'answered' 'article' 'attained' 'brave' 'cards'
 'clock' 'days' 'detailing' 'disk' 'especially' 'experiences' 'final'
 'floppy' 'floppies' 'functionality' 'heat' 'hour' 'keywords' 'knowledge'
 'lines' 'message' 'network' 'number' 'organization' 'oscillator'
 'posting' 'procedure' 'rated' 'reports' 'requested' 'send' 'shared'
 'sinks' 'souls' 'speed' 'subject' 'summary' 'summarizing' 'thanks'
 'upgrade' 'upgraded' 'usage'] 



In [20]:
type(train_vector_data)

scipy.sparse.csr.csr_matrix

Procedemos a elegir 3 mensajes de cada tipo

In [21]:
import random

In [22]:
#while the last element of the sum of all the previous elements (which is the sum of the total selectec texts) 
#    is less than the desired amount, it will keep choosing candidates

selected = [0]*len(train_data.target_names)

number_samples = 3

selected_index = [0]*len(train_data.target_names)*number_samples

while (np.cumsum(selected)[-1] < len(selected)*number_samples):

    #Indice aleatorio de la lista de mensajes
    index = random.randint(0, len(train_data.target))
    #cat = numero de la categoria a la que pertenece el índice elegido
    cat = train_data.target[index]
    #Comprobamos que no haya ya 3 elementos de la categoría a la que pertenezca el index y lo añadimos a la cuenta y a la lista de índices
    #La lista de índices estará ordenada por categorías
    if(selected[cat] < 3):
        selected_index[cat*number_samples + selected[cat]] = index
        selected[cat] += 1

In [23]:
selected

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [24]:
np.cumsum(selected)[-1]

60

In [25]:
len(selected_index)

60

In [26]:
selected_index

[5081,
 8005,
 10779,
 8738,
 7316,
 2302,
 11234,
 7404,
 2114,
 11095,
 9360,
 8447,
 6687,
 3797,
 10682,
 4510,
 5178,
 1980,
 9616,
 3777,
 2396,
 3730,
 3472,
 10079,
 7716,
 8789,
 3062,
 449,
 3619,
 9720,
 3997,
 6280,
 8984,
 7190,
 1660,
 8136,
 1488,
 1692,
 8141,
 8858,
 5301,
 663,
 4049,
 3665,
 11261,
 2012,
 5613,
 10446,
 10405,
 11111,
 434,
 8798,
 5828,
 4199,
 10240,
 4066,
 9485,
 1440,
 6706,
 7729]

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
type(train_vector_data[2252])

scipy.sparse.csr.csr_matrix

In [29]:
#indices = selected_index[0:3]
#print(indices)
#out1 = train_vector_data.tocsc()[indices,:]

[5081, 8005, 10779]


In [30]:
print(out1)

  (2, 787)	2
  (2, 897)	1
  (2, 973)	6
  (2, 977)	1
  (2, 1841)	1
  (2, 2073)	1
  (2, 2199)	1
  (2, 3749)	1
  (2, 4152)	1
  (0, 4971)	3
  (0, 5248)	2
  (2, 5483)	1
  (2, 5484)	1
  (2, 7502)	3
  (2, 7509)	1
  (2, 11112)	2
  (0, 11116)	1
  (2, 12837)	3
  (2, 14570)	1
  (0, 15392)	1
  (2, 17178)	1
  (2, 17191)	1
  (2, 17592)	1
  (0, 21356)	2
  (2, 21609)	1
  :	:
  (2, 445154)	2
  (2, 445296)	1
  (0, 448738)	2
  (1, 448742)	1
  (1, 449795)	2
  (2, 449820)	2
  (2, 450379)	1
  (0, 452830)	1
  (2, 452830)	1
  (2, 453624)	1
  (0, 457388)	1
  (2, 458353)	1
  (0, 459597)	1
  (0, 461144)	1
  (0, 461961)	1
  (2, 462783)	2
  (0, 462822)	1
  (1, 462822)	1
  (2, 463291)	1
  (2, 463400)	1
  (0, 463885)	1
  (2, 463885)	1
  (2, 463913)	1
  (2, 463924)	4
  (2, 463986)	2


In [31]:
cosSim1 = cosine_similarity(train_vector_data[2252], train_vector_data)

A ver, entonces, lo que creo que hay que hacer es:

    1) Hacer el cosine similarity para cada una de las 60 indices que hemos elegido.
    2) Agruparlos de 3 en 3 ya que es el número de muestras que nos dicen que cojamos (ponerlo como un parámetro)
    3) Ordenar los 20 arrays, en total cada uno tendrá (número de textos)*3 elementos
    4) Cuando tengamos todos los elementos ordenador hacemos lo de la exhaustividad (recall, es decir de entre todos los que han dado positivo (que se parecen al texto objetivo) cuantos son realmente de la misma categoría(True Pos/(True Pos + False Neg))) entre valores de 3 y 10. Eso significa que tenemos que ver para los mejores X valores (entre 3 y 10) cuantos son de la misma categoría que la clase que estamos comparando
    5) Hacer la media de los 

Nononono a ver

Lo que piden de:  la precisión de la lista de resultados con nivel de exhaustividad 3 y 10.

Lo que hay que hacer es: De los X valores que más se parezcan entre los X primeros recuperados. Por lo que símplemente es coger y decir: Si para un nivel de exhaustividad 7, cogemos los 7 elementos que más se parezcan a los textos que hemos metido, después comprobamos de esos 7 cuantos son de la clase a la que perteneces los mensajes seleccionados antes.

Entonces, después cogemos esos 8 resultados (los valores de 3 a 10) y hacemos la media. Eso para cada clase.

Así podemos ver cual es la clase que más parecidos tiene o algo así (?)


Y luego hay que hacer lo mismo pero con el TF-IDF

la duda que luego voy a preguntar es que si lo de exhaustivodad de nivel 5 por ejemplo, significa realmente que solo hay que coger los 5 elementos más TOP, me parece raro porque hay miles y miles y quedarse con tan pocos se me hace raro

Intentar hacer la movida con funciones para que quede guay y bien indicado

In [34]:
cosSim1.shape

(1, 11314)

In [131]:
cosSim[0][0]

1.000000000000001

In [35]:
cosSim2 = cosine_similarity(train_vector_data[2251], train_vector_data)

In [44]:
cosSim3 = cosine_similarity(train_vector_data[2250], train_vector_data)

In [107]:
cosSim = np.concatenate((cosSim1, cosSim2, cosSim3), axis=1)

In [97]:
type(cosSim)

numpy.ndarray

In [108]:
cosSim = -np.sort(-cosSim)

In [115]:
cosSimFinal = cosSim[0]

In [117]:
cosSimFinal.shape

(33942,)

In [249]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [303]:
#Funcion que le metes los índices seleccionados, el número de elementos por categoría y/o  el número de categorías
#Devuelve un array con un número de columnas igual al número de categorías y número de filas igual al
#    (número de textos - 1)*número de elementos elegidos por categoría - 3. El - 3 es para quitar las comparaciones de los textos
#    con ellos mismos. La lista está completamente ordenada de mayor a menor.

def cosSim_calc(indexes, elem, rows, numCat = None):
    print(indexes)
    print(len(indexes))
    
    cols = int(len(indexes)/elem)
    print(rows, cols)
    
    sim = [[0]*rows]*cols
    
    col = 0
    tempBatch_index = []
    #Para cada indice seleccionado hacemos la cosine_similarity
    for index in indexes:
        
        #Cogemos un grupo de "elem" elementos, de esta manera tendremos los indices con los que vamos a trabajar a continuación
        tempBatch_index.append(index)
        
        #Si ya hemos cogido los "elem" elementos, procedemos a usarlos
        if(len(tempBatch_index) == 3):
            #print(tempBatch_index)
            #Los agrupamos por categorías
            agg = []
            print(col)
            for i in range(elem):
                cosSim = cosine_similarity(train_vector_data[tempBatch_index[i]], train_vector_data)
                agg = np.concatenate((agg, cosSim[0]))
                #print(-np.sort(-agg))
                
            #Transformamos la representación de la información a dataFrames para poder mantener tener la categoría de cada elemento
            info = pd.DataFrame(agg, columns=['puntuacion'])
            #juntamos los indices para que coincidan con la información del DataFrame
            categorias = np.array([])
            for i in range(elem):
                categorias = np.append(categorias, train_data.target)
            
            info['categoria'] = categorias
            info = info.sort_values(by='puntuacion', ascending=False)
            info = info[elem:]
            sim[col] = info
            print(max(sim[col]['puntuacion']))
            #print(agg[:5])
            #sim[col] = -np.sort(-agg)
            #print(sim[col])
            #Ahora quitamos los elementos que son =1, es decir los que se han comparado con si mismos, que ahora están en las "elem"
            #    primeras posiciones
            #sim[col] = sim[col][elem:]
            
            #print(sim[col][:10])
            #print(max(sim[col]))
            #print(tempBatch_index)
            col+=1
            tempBatch_index = []

           
    return sim

In [304]:
cos_sim = cosSim_calc(selected_index, 3, len(train_data.data))

[5081, 8005, 10779, 8738, 7316, 2302, 11234, 7404, 2114, 11095, 9360, 8447, 6687, 3797, 10682, 4510, 5178, 1980, 9616, 3777, 2396, 3730, 3472, 10079, 7716, 8789, 3062, 449, 3619, 9720, 3997, 6280, 8984, 7190, 1660, 8136, 1488, 1692, 8141, 8858, 5301, 663, 4049, 3665, 11261, 2012, 5613, 10446, 10405, 11111, 434, 8798, 5828, 4199, 10240, 4066, 9485, 1440, 6706, 7729]
60
11314 20
0
0.6223561592368898
1
0.849861767061256
2
0.7727272727272726
3
0.9919692415546342
4
0.36548694232390355
5
0.7607159437763318
6
0.4714045207910317
7
0.8624430703157185
8
0.8563488385776753
9
0.7537346628814319
10
0.7469377433031636
11
0.6155380992480795
12
0.710046946804692
13
0.8421978967542583
14
0.9999999999999996
15
0.6297144832524354
16
0.4756707120084505
17
0.8451542547285162
18
0.8175191193132879
19
0.43069868239124


In [309]:
cos_sim

[       puntuacion  categoria
 647      0.622356        0.0
 12923    0.568552        0.0
 23765    0.556685        0.0
 29882    0.517773        0.0
 22943    0.476086        0.0
 ...           ...        ...
 26718    0.001406        2.0
 27143    0.001107        2.0
 27210    0.001051       10.0
 22772    0.000752        2.0
 24169    0.000676        2.0
 
 [33939 rows x 2 columns],        puntuacion  categoria
 1616     0.849862        1.0
 18737    0.836619        1.0
 12092    0.820610        1.0
 12326    0.778499        1.0
 17800    0.765532        1.0
 ...           ...        ...
 12855    0.004934        2.0
 19833    0.004839        6.0
 1541     0.004794        2.0
 20726    0.004029       10.0
 9412     0.003914       10.0
 
 [33939 rows x 2 columns],        puntuacion  categoria
 4615     0.772727        2.0
 30937    0.690980        2.0
 29711    0.569666        2.0
 28042    0.528997        2.0
 31308    0.528813        2.0
 ...           ...        ...
 19833    0.00

In [306]:
print(max(cos_sim[13]['puntuacion'].values))

0.8421978967542583


In [310]:
for elem in cos_sim:
    print(max(elem['puntuacion'].values))

0.6223561592368898
0.849861767061256
0.7727272727272726
0.9919692415546342
0.36548694232390355
0.7607159437763318
0.4714045207910317
0.8624430703157185
0.8563488385776753
0.7537346628814319
0.7469377433031636
0.6155380992480795
0.710046946804692
0.8421978967542583
0.9999999999999996
0.6297144832524354
0.4756707120084505
0.8451542547285162
0.8175191193132879
0.43069868239124
